In [44]:
from coal_mapper_dataset import *
from coal_mapper import *
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import plotly_express as px
from plotly.subplots import make_subplots

In [45]:
total = pd.read_excel('AAtotal.xlsx')
del total['level_0']


In [105]:
gen = pd.read_excel('egrid 2005-2020/egrid2020_data.xlsx', 'GEN20', header = 1).reset_index()
indivL = gen['ORISPL'].tolist()
plantL = total['ORISPL'].tolist() ####to Edit
gen_list = []
gen2 = gen
count = 0
for x in indivL:
    if x in plantL:
        count +=1
        gen_list.append(x)
gen2 = gen2[gen2['ORISPL'].isin(gen_list)]
# In[16]:
def removeNonCoal(gen2):
    gen2 = gen2[(gen2.FUELG1 != 'WND') & (gen2.FUELG1 != 'WDS') & (gen2.FUELG1 != 'WAT') & (gen2.FUELG1 != 'SUN') & (gen2.FUELG1 != 'RFO') & (gen2.FUELG1 != 'NG') & (gen2.FUELG1 != 'JF') & (gen2.FUELG1 != 'DFO') & (gen2.FUELG1 != 'NUC') & (gen2.FUELG1 != 'BLQ') ]
    return gen2
gen2 = removeNonCoal(gen2)

gen2 = gen2[gen2['GENYRRET']<=2030]
gen2 = gen2[gen2['GENYRRET']>2020]
gen2 = gen2[['ORISPL', 'GENYRRET']]
tot = total.merge(gen2, how='left', left_on='ORISPL', right_on='ORISPL')

def plannedRet (id):
    if id > 2020 and id < 2026:
        return 1
    elif id > 2025:
        return 2
    else:
        return 0

tot['ret'] = tot['GENYRRET'].apply(plannedRet)
tot = tot.groupby('ORISPL').mean().reset_index()
tot=tot[['ORISPL', 'ret']]

retTotal = total.merge(tot, how='left', on='ORISPL')

In [47]:
total2 = total.sort_values('cluster')
total3 = total.groupby('cluster').mean()

In [48]:
from sklearn.preprocessing import MinMaxScaler

def df_minMaxScaler(d, lister):
    for PLGENACL in lister:
        d[PLGENACL] = np.log(d[PLGENACL])
        d[PLGENACL].replace([np.inf, -np.inf], 0, inplace=True)
        d[PLGENACL] = MinMaxScaler().fit_transform(d[PLGENACL].values.reshape(-1, 1))
        #d[PLGENACL] = (d[PLGENACL]-d[PLGENACL].min())/(d[PLGENACL].max()-d[PLGENACL].min())
    return d

scaledDF = df_minMaxScaler(total2, ['PLGENACL', 'AverageAge', 'forwardCosts', 'PLSO2AN', 'affectedDemographicIndex',  'CO2limitsOppose'])
scaled_df = scaledDF.groupby('cluster').mean().reset_index()

/Users/gathrid/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning:

divide by zero encountered in log

/Users/gathrid/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning:

divide by zero encountered in log



In [49]:
clusterList = [['p'], ['j', 'e', 'l', 'a'], ['g', 'o', 'b'], ['c'], ['i'], ['k', 'h', 'd', 'm', 'n', 'f']]

def createSuperCluster(id):
    for clusterGroup in clusterList:
        for clusterLetter in clusterGroup:
            if id==clusterLetter:
                return clusterList.index(clusterGroup)+1

In [50]:
total['superCluster'] = total['cluster'].apply(createSuperCluster)
total2['superCluster'] = total2['cluster'].apply(createSuperCluster)
retTotal['superCluster'] = retTotal['cluster'].apply(createSuperCluster)

to be done: 
- individual colors per superCluster, make sure these colors can be used across every figure to ensure continunity
- pivot bars
- add other important criteria making more figures in the combined plot
– maybe put all in one plot? categorically?

point of these specific barplot summary figures: while we are decommissioning plants, the rate at which we are doing so is way to slow. This paper aims to provide a framework for how to target and hasten the retirement of plants within each cluster, assuming that because they have similar attributes, the policy/lobbying approach to targeting these plants can be very similar. 

info for the paper: the rest of the figures ignore plants that already have planned retirements on the books, no need to spend political capitol targeting these plants.

goal of topology: levering a datascience tool to create groupings from a high-level standpoint. These are merely descriptive statistics, aiming to illuminate th insights provided by the mapper in a more traditional and usable light.

,Unnamed: 0,index,Year,ORISPL,PSTATABB,LAT,LON,PNAME,FIPSST,FIPSCNTY,...,impact_score,AverageAge,forwardCosts,Confidence Tier,_merge,numPlants,p_num,cluster,ret,superCluster
2,2,67.0,2020,1893,MN,47.261100,-93.652800,Boswell Energy Center,27,61,...,11211.142141,41.765854,28.353251,1.0,both,1,0,a,1,2
7,7,195.0,2020,8023,WI,43.486400,-89.420300,Columbia,55,21,...,11485.699424,42.500000,35.259980,1.0,both,1,0,a,1,2
15,15,84.0,2020,2721,NC,35.220000,-81.759400,Cliffside,37,45,...,11342.708341,23.229990,36.618062,2.0,both,1,0,a,1,2
19,19,145.0,2020,6085,IN,41.216400,-87.026100,R M Schahfer Generating Station,18,73,...,11224.070821,38.436503,48.717695,1.0,both,1,0,a,1,2
21,21,160.0,2020,6166,IN,37.925600,-87.037200,Rockport,18,147,...,12570.736599,32.500000,61.341615,1.0,both,1,0,a,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,246,NaN,2020,1832,MI,42.692222,-84.657222,Erickson,26,45,...,13600.911555,46.000000,43.997064,2.0,both,1,14,o,1,3
249,249,NaN,2020,4050,WI,43.715142,-87.705863,Edgewater (4050),55,117,...,11547.995021,34.000000,43.554504,1.0,both,1,14,o,1,3
263,263,NaN,2020,2832,OH,39.112800,-84.803600,Miami Fort Power Station,39,61,...,14603.371863,42.499193,26.221738,4.0,both,1,15,p,1,1
265,265,NaN,2020,6019,OH,38.867500,-84.228900,W H Zimmer Generating Station,39,25,...,13063.758954,28.000000,35.834081,6.0,both,1,15,p,1,1


In [95]:
fig = make_subplots(rows=2, cols=3, specs=
        [[{}, {}, {}],
        [{}, {}, {}]])

dff = retTotal.groupby('superCluster').sum()
dff2 = retTotal[retTotal['ret']==1].groupby('superCluster').sum()
fig.add_trace(go.Bar(x=dff['affectedDemographicIndex'], marker=dict(color=px.colors.qualitative.Plotly_r[2])), row=1, col=1)
fig.add_trace(go.Bar(x=dff2['affectedDemographicIndex'], marker=dict(color='red')), row=1, col=1)


dff = retTotal.groupby('superCluster').sum() ## Overlay of amount to be retired by 2030
dff2 = retTotal[retTotal['ret']==1].groupby('superCluster').sum()
fig.add_trace(go.Bar(x=dff['PLGENACL']), row=2, col=1)
fig.add_trace(go.Bar(x=dff2['PLGENACL']), row=2, col=1)


dff = retTotal.groupby('superCluster').sum()
dff2 = retTotal[retTotal['ret']==1].groupby('superCluster').sum()
fig.add_trace(go.Bar(x=dff['forwardCosts']), row=1, col=2)
fig.add_trace(go.Bar(x=dff2['forwardCosts']), row=1, col=2)
fig.update_layout(barmode='stack')

In [492]:
fig=go.Figure()

fig.add_trace(go.Histogram(x=retTotal['PLGENACL'], name = 'Coal Plants in 2020', marker_color='grey', xbins=dict(size=400000)))
fig.add_trace(go.Histogram(x=retTotal[retTotal['ret']>0]['PLGENACL'], name = 'Plants Retired by 2030', marker_color='lightgreen', xbins=dict(size=400000)))
fig.add_trace(go.Histogram(x=retTotal[retTotal['ret']==1]['PLGENACL'], name = 'Plants Retired by 2025', marker_color='green', xbins=dict(size=400000)))
fig.update_layout(barmode='overlay', legend={'traceorder':'normal'})


In [192]:
fig = go.Figure()

vList = [retTotal[retTotal['ret']==0]['numPlants'].sum(), retTotal[retTotal['ret']>1]['numPlants'].sum(), retTotal[retTotal['ret']==1]['numPlants'].sum()]
labels=['Total Active Generation (MWh/yr)', 'Total Generation Retired by 2025', 'Total Generation Retired by 2030']
fig.add_trace(go.Pie(values = vList, marker_colors=['grey', 'lightgreen', 'green']))
fig

In [509]:
fig = make_subplots(rows=1, cols=3, subplot_titles=("Planned Coal Plant Retirements", "Generation (MWh/yr in 2020) of Retiring Plants", "Forward Costs ($/MWh in 2020) of Retiring Plants"), specs=
        [[{}, {"type": "Pie"}, {"type": "Pie"}]]
)
fig.add_trace(go.Histogram(x=retTotal['PLGENACL'], name = f"Coal Plants in 2020 - {retTotal['numPlants'].count()} Plants", xbins=dict(size=837500), marker_color='grey'), 1, 1)
fig.add_trace(go.Histogram(x=retTotal[retTotal['ret']>0]['PLGENACL'], name = f"Plants Retiring by 2030 - {retTotal[retTotal['ret']>1]['numPlants'].count()} Plants", xbins=dict(size=482500), marker_color='lightgreen'), 1, 1)
fig.add_trace(go.Histogram(x=retTotal[retTotal['ret']==1]['PLGENACL'], name = f"Plants Retiring by 2025 - {retTotal[retTotal['ret']==1]['numPlants'].count()} Plants", xbins=dict(size=482500), marker_color='green'), 1, 1)

vList = [retTotal[retTotal['ret']==0]['PLGENACL'].sum(), retTotal[retTotal['ret']>1]['PLGENACL'].sum(), retTotal[retTotal['ret']==1]['PLGENACL'].sum()]
fig.add_trace(go.Pie(values = vList, marker_colors=['grey', 'lightgreen', 'green'], hole=0.4, direction ='clockwise', showlegend=False), 1, 2)

vList = [retTotal[retTotal['ret']==0]['forwardCosts'].sum(), retTotal[retTotal['ret']>1]['forwardCosts'].sum(), retTotal[retTotal['ret']==1]['forwardCosts'].sum()]
fig.add_trace(go.Pie(values = vList, marker_colors=['darkgrey', 'lightgreen', 'green'], hole=0.4, showlegend=False), 1, 3)

fig['layout']['xaxis']['title']='Annual Generation (MWh/yr)'
fig['layout']['yaxis']['title']='Number of Plants'
fig['layout']['xaxis']['title']['font'] = dict(size=9)
fig['layout']['yaxis']['title']['font'] = dict(size=9)

fig.update_annotations(font_size=12)
fig.update_layout(barmode='overlay', legend=dict(traceorder='reversed', yanchor="bottom", y=-.3, xanchor = 'left', x=.19, orientation="h"), font = dict(size=12))
#legend=dict(yanchor="top", y=0.9, xanchor="left", x=0.4)


In [52]:


fig = make_subplots(
    rows=2, cols=3,
    vertical_spacing=0.3,
    specs=
        [[{}, {}, {}],
        [{}, {}, {}]]
    #subplot_titles=("", "", "", "Average Cluster Traits")
)

labelsList = ["Highest SO2 Emitters", 'Large Env. Impacts, Pro-Coal Public Sentiment', 'High DAC & Community Impacts', 'Minimal DAC and Community Impacts', 'Newer Plants, Anti-Coal Public Sentiment', 'Un-Grouped']


for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['PLGENACL'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i])), row=1, col=1)

for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['AverageAge'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i]), showlegend=False), row=1, col=2)

for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['forwardCosts'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i]), showlegend=False), row=1, col=3)

for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['PLSO2AN'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i]), showlegend=False), row=2, col=1)

for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['affectedDemographicIndex'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i]), showlegend=False), row=2, col=2)

for i in reversed(range(1, 6)):
    fig.add_trace(go.Violin(x=total[total['superCluster']==i]['CO2limitsOppose'], name=labelsList[i-1], marker=dict(color=px.colors.qualitative.Plotly_r[i]), showlegend=False), row=2, col=3)

fig['layout']['xaxis']['title']='Annual Generation (MWh/yr)'
fig['layout']['xaxis2']['title']='Average Age'
fig['layout']['xaxis3']['title']='Forward Costs ($/MWh)'
fig['layout']['xaxis4']['title']='SO2 Emissions (tons/yr)'
fig['layout']['xaxis5']['title']='Affected DAC within 3 miles'
fig['layout']['xaxis6']['title']='Public Support for Coal (%)'

fig.update_traces(orientation='h', side='positive', width=3, points=False)
fig.update_layout(legend_traceorder="reversed")
fig.update_yaxes(visible=False, showticklabels=False)




In [53]:
scaledDF = df_minMaxScaler(total2.groupby('cluster').mean(), ['PLGENACL', 'AverageAge', 'forwardCosts', 'PLSO2AN', 'affectedDemographicIndex',  'CO2limitsOppose'])
vis_df = scaledDF[['superCluster','PLGENACL', 'AverageAge', 'forwardCosts', 'PLSO2AN', 'affectedDemographicIndex', 'CO2limitsOppose']].groupby('superCluster').mean()

/Users/gathrid/opt/anaconda3/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning:

divide by zero encountered in log



In [54]:
fig = go.Figure()

for i in range(0, 6):
    fig.add_trace(go.Scatterpolar(
      r=list(vis_df.iloc[i]),
      theta=list(['Annual Generation', 'Average Age', 'Forward Costs', 'SO2 Emissions', 'Affected DAC within 3 miles', 'Public Support for Coal Power']),
      fill='toself', 
      line_color=px.colors.qualitative.Plotly_r[i+1],
      name = labelsList[i]
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  showlegend=True,
  legend_title='Cluster'
)

fig.show()

In [55]:
fig = make_subplots(
        rows=2, cols=3,
        vertical_spacing=0.2,
        horizontal_spacing=0.1,
        specs=
            [[{"type": "Scatterpolar",}, {"type": "Scatterpolar",}, {"type": "Scatterpolar",}],
            [{"type": "Scatterpolar",}, {"type": "Scatterpolar",}, {"type": "Scatterpolar",}]],
        #subplot_titles=("", "", "", "Average Cluster Traits")
    )

row1=1
col1=1

for i in range(0, 6):
    fig.add_trace(go.Scatterpolar(
      r=list(vis_df.iloc[i]),
      theta=list(['Annual Generation', 'Average Age', 'Forward Costs', 'SO2 Emissions', 'Affected DAC within 3 miles', 'Public Support for Coal Power']),
      fill='toself', 
      line_color=px.colors.qualitative.Plotly_r[i+1],
      name = labelsList[i]
    ), row=row1, col=col1)
    col1+=1
    if col1==4:
       col1=1
    if col1==1:
       row1=2

fig.update_layout(
  polar1=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar2=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar3=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar4=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar5=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar6=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  showlegend=True,
  legend_title='Cluster'
)
fig.update_layout(width=1200, height = 500, font = dict(size=9), legend = dict(valign = 'bottom'), showlegend=False)
fig.show()

In [56]:

fig = go.Figure()
for item in ['PLGENACL', 'AverageAge', 'forwardCosts', 'affectedDemographicIndex', 'PLSO2AN', 'CO2limitsOppose']:
    for i in range(1, 7):
        fig.add_trace(go.Box(y=total[item][total['superCluster']==i], name=labelsList[i-1]))
fig

In [512]:
fig = make_subplots(
        rows=2, cols=3,
        vertical_spacing=0.3,
        specs=
            [[{}, {}, {}],
            [{}, {}, {}]],
        #subplot_titles=("", "", "", "Average Cluster Traits")
    )
row1=1
col1=1

for item in ['PLGENACL', 'AverageAge', 'forwardCosts', 'affectedDemographicIndex', 'PLSO2AN', 'CO2limitsOppose']:
    for i in range(1, 7):
        fig.add_trace(go.Box(y=total[item][total['superCluster']==i], name=labelsList[i-1], showlegend=False, line_color = px.colors.qualitative.Plotly_r[i]), row=row1, col=col1)
    col1+=1
    if col1==4:
        col1=1
    if col1==1:
        row1=2


fig.update_xaxes(visible=False, showticklabels=False)
fig.update_layout(font = dict(size=9), )

In [566]:
fig = make_subplots(
        rows=4, cols=3,
        vertical_spacing=0.1,
        horizontal_spacing=0.05,
        specs=
            [[{"type": "Scatterpolar",}, {"type": "Scatterpolar",}, {"type": "Scatterpolar",}],
            [{"type": "Scatterpolar",}, {"type": "Scatterpolar",}, {"type": "Scatterpolar",}],
            [{}, {}, {}],
            [{}, {}, {}]],
        subplot_titles=("", "", "", "", "", "", "Annual Generation (MWh/yr)", 'Average Age', 'Forward Costs ($/MWh)', 'Affected DAC within 3 Miles', 'Annual SO2 Emissions (tons/yr)', 'Public Support for Coal Power')
    )

row1=1
col1=1

for i in range(0, 6):
    fig.add_trace(go.Scatterpolar(
      r=list(vis_df.iloc[i]),
      theta=list(['Annual Generation', 'Average Age', 'Forward Costs', 'SO2 Emissions', 'Affected DAC within 3 miles', 'Public Support for Coal Power']),
      fill='toself', 
      line_color=px.colors.qualitative.Plotly_r[i+1],
      name = labelsList[i]
    ), row=row1, col=col1)
    col1+=1
    if col1==4:
       col1=1
    if col1==1:
       row1=2


row1=3
col1=1

for item in ['PLGENACL', 'AverageAge', 'forwardCosts', 'affectedDemographicIndex', 'PLSO2AN', 'CO2limitsOppose']:
    for i in range(1, 7):
        fig.add_trace(go.Box(y=total[item][total['superCluster']==i], name=labelsList[i-1], showlegend=False, line_color = px.colors.qualitative.Plotly_r[i]), row=row1, col=col1)
    col1+=1
    if col1==4:
        col1=1
    if col1==1:
        row1=4


fig.update_xaxes( showticklabels=False)
fig.update_layout(font = dict(size=9), )


fig.update_layout(
  polar1=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar2=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar3=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar4=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar5=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  polar6=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1],
      showticklabels=False
    ),
  ),
  showlegend=True,
  legend_title='Group'
)

fig.update_annotations(font_size=12)
fig.update_layout(width=1200, height = 700, font = dict(size=10),
legend=dict(traceorder='normal',  orientation="h", yanchor = 'bottom', y=-0.15, xanchor = 'center', x=0.5))
fig.show()

In [69]:
(px.histogram(retTotal, x="affectedDemographicIndex", color="ret",
            facet_col="superCluster"))

In [59]:
import plotly.express as px

df = total

(px.scatter(df, x="affectedDemographicIndex", y="forwardCosts", color="cluster",
            facet_col="superCluster", 
            title="Chaining Multiple Figure Operations With A Plotly Express Figure")
 .update_layout(title_font_size=24)
 .update_xaxes(showgrid=False)
 .update_traces(
     line=dict(dash="dot", width=4),
     selector=dict(type="scatter", mode="lines"))
).show()

In [60]:

bardf = vis_df.reset_index()
pivotedDF = bardf.pivot_table(columns='superCluster')
del pivotedDF[6]

fig = go.Figure()

for i in range(0, 6):
    fig.add_trace(go.Bar(x=labelsList, y=pivotedDF.iloc[i], name = pivotedDF.index[i]))

# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [61]:
fig = go.Figure()
for i in range(1, 7):
    fig.add_trace(go.Box(y=total['PLGENACL'][total['superCluster']==i]))
fig

In [62]:
fig = go.Figure()

for item in ['PLGENACL', 'AverageAge', 'forwardCosts', 'PLSO2AN', 'affectedDemographicIndex',  'CO2limitsOppose']:
    for i in range(1, 6):
        if i%2==0:
            var3='negative'
        else:
            var3='positive'
        fig.add_trace(go.Violin(y=total2[item][total2['superCluster']==i], name =item, side=var3, showlegend=False,
                                    line_color=px.colors.qualitative.Plotly_r[i])
                        )

#fig.update_yaxes(visible=False)

fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0,  violingroupgap=0, violinmode='overlay')
fig.show()

In [306]:
import plotly.graph_objects as go

labels = ["Highest SO2 Emitters", 'Large Env. Impacts, Pro-Coal Public Sentiment', 'High DAC & Community Impacts', 'Minimal DAC and Community Impacts', 'Newer Plants, Anti-Coal Public Sentiment', 'Un-Grouped']


fig = go.Figure()

fig.add_trace(go.Pie(labels=labels, values=mapdf['PLGENACL'], hole=0.1, name="Generation", marker_colors=['#FF97FF','#B6E880','#FF6692','#19D3F3','#FFA15A','#AB63FA',]),
)


# Use `hole` to create a donut-like pie chart
fig.update_traces(hoverinfo="label+percent+name")

fig.update_layout(title_text="",
    annotations=[dict(text='MWh/yr', x=0.5, y=0.5, font_size=10, showarrow=False)])
    
fig.show()

In [293]:
mapdf = (total.groupby('superCluster')['PLGENACL', 'PLSO2AN', 'forwardCosts', 'affectedDemographicIndex', 'impact_score', 'numPlants'].sum()/total[['PLGENACL', 'PLSO2AN', 'forwardCosts', 'affectedDemographicIndex', 'impact_score', 'numPlants']].sum())*100
#mapdf=mapdf[:-1]

x1=['Annual Generation', 'Total SO2 Released', 'Forward Costs', 'Affected DAC within 3 Miles', 'Impact Score', 'Number of Plants']
#y1 = ['Minimal DAC and Community Impacts', 'Newer Plants, Anti-Coal Public Sentiment', 'High DAC & Community Impacts',   "Highest SO2 Emitters", 'Large Env. Impacts, Pro-Coal Public Sentiment']
y1 = [  'Minimal DAC and Community Impacts', 'Un-Grouped', 'High DAC & Community Impacts', 'Newer Plants, Anti-Coal Public Sentiment',  "Highest SO2 Emitters", 'Large Env. Impacts, Pro-Coal Public Sentiment',]
z1 = mapdf.sort_values('PLGENACL').values.tolist()
tList1 = mapdf.sort_values('PLGENACL').round(2).values.tolist()

fig = go.Figure()
fig.add_trace(go.Heatmap(x=x1, y=y1, z=z1, colorscale = 'rdylgn_r', text=tList1, texttemplate=("%{text}"+"%")))

fig.update_xaxes(tickangle=45)

fig.update_layout(font=dict(
        #family="Courier New, monospace",
        size=10,
        #color="RebeccaPurple"
    ),
    width=1000,
    height=700)

fig.show()

<ipython-input-293-2161e558de58>:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

<ipython-input-293-2161e558de58>:2: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



In [ ]:
fig =go.Figure(go.Sunburst(
    #labels=["Type of Admission", "Other Medical", "Post Surgery", "Adult", "Pediatric", "Readmission", "Post Cath", "adult", "pediatric", "Pre-Operative"],
    #parents=["","Type of Admission","Type of Admission","Post Surgery","Post Surgery", "Type of Admission","Type of Admission","Post Cath","Post Cath","Type of Admission"],))

    labels = [' ', "High SO2 Emitters", 'Large Env Impacts', 'High DAC & Community Impacts', 'Minimal DAC and Community Impacts', 'Newer Plants, Anti-Coal Public Sentiment', 'Un-Grouped', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P'],
    parents = ['', ' ', ' ',' ', ' ',' ', ' ', 'Large Env Impacts', 'High DAC & Community Impacts', 'Minimal DAC and Community Impacts', 'Un-Grouped', 'Large Env Impacts', 'Un-Grouped', 'High DAC & Community Impacts', 'Un-Grouped', 'Newer Plants, Anti-Coal Public Sentiment', 'Large Env Impacts', 'Un-Grouped', 'Large Env Impacts', 'Un-Grouped', 'Un-Grouped','High DAC & Community Impacts', "High SO2 Emitters"],
    values = l32
    ))

fig.update_layout(margin = dict(t=0, l=0, r=0, b=0))

In [ ]:
mapdf2 = (total.groupby('cluster')['PLGENACL', 'PLSO2AN', 'forwardCosts', 'affectedDemographicIndex', 'impact_score', 'numPlants'].sum()/total[['PLGENACL', 'PLSO2AN', 'forwardCosts', 'affectedDemographicIndex', 'impact_score', 'numPlants']].sum())*100
mapdf2 = mapdf2.reset_index()

super_cluster_list = [['p'], ['j', 'e', 'l', 'a'], ['g', 'o', 'b'], ['c'], ['i'], ['k', 'h', 'd', 'm', 'n', 'f']]
def label_superCluster(id):
    for i in range(0, len(super_cluster_list)):
        for n in super_cluster_list[i]:
            if id==n:
                return i+1

mapdf2['superCluster'] = mapdf2['cluster'].apply(label_superCluster)
list2 = mapdf['PLGENACL'].to_list()
list2.append(mapdf2['PLGENACL'].to_list())
list2.insert(0, 10)


In [ ]:
l32 = [0, 17.01060926139145,
 65.9362700172107,
 6.551072564698471,
 1.7315916346238678,
 5.555969167180564,
 3.214487354894947, 19.95139905714295, 4.28974885312082,
 1.7315916346238678,
 0.38912395379144366,
 5.6864816069579245,
 1.7767546388448716,
 0.8741336362064506,
 0.15260734358437414,
 5.555969167180564,
 6.2183049428543375,
 0.026857163194956846,
 34.08008441025549,
 0.39281021094280977,
 0.47633404453649,
 1.3871900753711974,
 17.01060926139145]

In [ ]:

import plotly.express as px
# fig = px.sunburst(mapdf2, path=['superCluster', 'cluster'], values='PLGENACL')
# fig.show()

fig = px.sunburst(mapdf2, path=['superCluster', 'cluster'], values='PLSO2AN')

# plotly graph_objects figure
fig2 =go.Figure(go.Sunburst(
    labels=fig['data'][0]['labels'].tolist(),
    parents=fig['data'][0]['parents'].tolist(),
    values=fig['data'][0]['values'].tolist(),
    #ids=fig['data'][0]['ids'].tolist(),
    #domain={'x': [0.0, 1.0], 'y': [0.0, 1.0]}
))

# fig2 =go.Figure(go.Sunburst(
#     labels=mapdf2['cluster'].tolist(),
#     parents=mapdf2['superCluster'].tolist(),
#     values=mapdf2['PLGENACL'].tolist(),
#     ids=fig['data'][0]['ids'].tolist(),
#     domain={'x': [0.0, 1.0], 'y': [0.0, 1.0]}
# ))

# fig2.update_layout(width=1500,
#     height = 1000,)
fig2.show()

In [578]:
fig = go.Figure()

clusterList = [['p'], ['j', 'e', 'l', 'a'], ['g', 'o', 'b'], ['c'], ['i'], ['k', 'h', 'd', 'm', 'n', 'f']]

outerDF = retTotal.groupby('superCluster')['PLGENACL'].sum().reset_index()
outerDF22 = retTotal.groupby('cluster')['PLGENACL'].sum()
outerDF22 = outerDF22.reindex(['p','j', 'e', 'l', 'a', 'g', 'o', 'b', 'c', 'i', 'k', 'h', 'd', 'm', 'n', 'f']).reset_index()


fig.add_trace(go.Pie(values=outerDF['PLGENACL'], hole=0, sort=False, textposition='inside', name = 'Hellothere',
insidetextorientation='radial', pull=[0,0, 0, 0, 0, 0.2],
marker=dict(colors=px.colors.qualitative.Plotly_r[1:7], line=dict(color='white', width=1)), 
#texttemplate=(["High SO2 Emitters", 'Large Env Impacts', 'High DAC Impacts', 'Minimal DAC Impacts', 'Newer Plants, Anti-Coal Public Sentiment', 'Un-Grouped      ',])
),
)

fig.add_trace(go.Pie(values=outerDF22['PLGENACL'], textinfo='text', hole=0.93, sort=False, textposition='outside', showlegend=False,
marker=dict(colors=px.colors.sequential.Greys,line=dict(color='black', width=0.5)), pull=[0,0, 0, 0, 0, 0, 0, 0, 0, 0, 0.1, 0.1, 0.1,0.1, 0.1, 0.1],
texttemplate=(['p','j', 'e', 'l', 'a', 'g', 'o', 'b', 'c', 'i', 'k', 'h', 'd', 'm', 'n', 'f'])
))

fig.update_layout( margin=dict(t=0, b=0, l=0, r=0), font=dict(size=10))

In [321]:
outerDF[outerDF['cluster'].isin(['p'])]['PLGENACL']

15    1.298748e+08
Name: PLGENACL, dtype: float64

In [374]:
outerDF22[0]

129874846.44199999